# Seattle Lidar

## Visualize Lidar Scattered Point Elevation Data

This notebook uses Datashader to visualize Lidar elevation data from [the Puget Sound Lidar consortium](http://pugetsoundlidar.ess.washington.edu/), a source of Lidar data for the Puget Sound region of the state of Washington, U.S.A.

### Lidar Elevation Data

Example X,Y,Z scattered point elevation data from the unpacked 7zip files (unpacked as .gnd files)

```
! head ../data/q47122d2101.gnd
```
```
X,Y,Z
1291149.60,181033.64,467.95
1291113.29,181032.53,460.24
1291065.38,181035.74,451.41
1291113.16,181037.32,455.51
1291116.68,181037.42,456.20
1291162.42,181038.90,467.81
1291111.90,181038.15,454.89
1291066.62,181036.73,451.41
1291019.10,181035.20,451.64
```

The Seattle area example below loads 25 `.gnd` elevation files like the one above. We'll download, cache and read the data using `intake`. 

In [ ]:
import intake

cat = intake.open_catalog('../catalog.yml')
list(cat)

In [ ]:
lidar = cat.seattle_lidar()
df = lidar.to_dask()
df.head()

### Geographic Metadata

Since the data are geographic, we need to know the coordinate reference system (CRS) of the X and Y. All the geographic metadata is stored in the data source entry in the intake catalog.

In [ ]:
from pyproj import Proj, transform

In [ ]:
lidar.metadata['crs']

In [ ]:
washington_state_plane = Proj(init='epsg:2855')   # Washington State Plane North EPSG code
web_mercator = Proj(init='epsg:3857')             # Mercator projection EPSG code

In [ ]:
FT_2_M = 0.3048  

def convert_coords(df):
    lon, lat = transform(washington_state_plane, web_mercator, df.X.values * FT_2_M, df.Y.values * FT_2_M)
    df['meterswest'], df['metersnorth'] = lon, lat 
    return df[['meterswest', 'metersnorth', 'Z']]

Try out the convert_coords function on a subset of the data

In [ ]:
convert_coords(df.head())

### Convert the coordinates

Since our real dataset is large and partitioned using dask, we need to think about how to apply the convert_coords function to our data. 

In [ ]:
import dask.distributed
import dask.delayed

In [ ]:
dask.distributed.Client()

Explore the task graph to figure out a performant way to split up the coords conversion. First we'll try with using `dask.delayed`.

In [ ]:
dask.delayed(convert_coords)(df) \
    .visualize()

We can see that even though we thought `dask.delayed` would help, in actuality we would be requiring all the processes to be done first and then the conversion would happen on the whole dataset in one go. Another approach would be to use `dask.map_partitions` to do the conversion on each piece of the data. 

In [ ]:
df_merc = df.map_partitions(convert_coords)
df_merc.visualize()

Now that we have set up the task graph, we can use `df_merc` directly to do the computations on the fly. However, since this dataset fits in memory on my computer, I will do the computation and keep the output in memory for quick use when plotting. 

**NOTE:** This next cell takes about a minute to run. Take a look at the `dask` dashboard at the location specified above to watch the task progression.

In [ ]:
%time dataset = df_merc.compute()

If all the data doesn't fit in memory on your machine, try downsampling the data from each file to only keep 1/100th of the total data. To avoid unnecessary computation, it is better to do the downsampling first and _then_ convert the coordinates.

In [ ]:
small = df.map_partitions(lambda x: convert_coords(x.sample(frac=0.01)))
small.visualize()

In [ ]:
%time small_dataset = small.compute()

### Plot

In [ ]:
import geoviews as gv
import holoviews as hv
from holoviews import opts
from holoviews.operation.datashader import datashade, rasterize, rd

hv.extension('bokeh')

To simplify the exploration of the time required to display different data, define a function that accepts a regular `pandas` dataframe or a `dask` delayed dataframe. 

In [ ]:
def plot(data, **kwargs):
    """Plot point elevation data, rasterizing by mean elevation"""
    points = hv.Points(data, kdims=['meterswest', 'metersnorth'], vdims=['Z'])
    image_opts = opts.Image(tools=['hover'], cmap='blues_r', colorbar=True,
                            width=800, height=800, xaxis=None, yaxis=None)
    return rasterize(points, aggregator=rd.mean('Z'), precompute=True, **kwargs).options(image_opts)

We'll also declare a tiler to use for background imagery.

In [ ]:
tiles = gv.tile_sources.EsriImagery()

Then we will construct the plot out of rasterized point data and tiles. 

In [ ]:
tiles * plot(small_dataset)

### Benchmarking

In [ ]:
%time display(tiles * plot(small_dataset))

In [ ]:
%time display(tiles * plot(dataset))

In [ ]:
%time display(tiles * plot(df_merc))

## Visualize Raster of Point Data

If we compute a raster of the point data then we don't need to store as much data in memory, which should allow faster interactions, and allow use with lots of other tools. 

In [ ]:
%time raster = plot(dataset, dynamic=False, width=1000, height=1000).data

The result is an `xarray.Dataset` with x and y coordinates and a 2D array of Z values.

In [ ]:
raster.metersnorth[1] - raster.metersnorth[0]

With these data we can use the geo tools in datashader to compute and visualize the elevation using hillshading for instance. See [Datashader User Guide #8](https://github.com/pyviz/datashader/blob/master/examples/user_guide/8_Geography.ipynb) for more datashader geo tooling. 

In [ ]:
from datashader.geo import hillshade

In [ ]:
illuminated = hillshade(raster.Z)
hv_shaded = hv.Image((raster.meterswest, raster.metersnorth, illuminated))

In [ ]:
tiles * rasterize(hv_shaded.options(opts.Image(cmap='blues', height=600, width=600)))

## Ideas for future work

It'd be nice to have a `rasterio` writer from `xarray` so that we could easily write chunked geotiffs from `xarray` objects.

Something like:

```python
raster.to_rasterio(path=None, mode='w', compute=True)
```